# get_comments

* 歌動画に絞ってコメントを取得します

In [1]:
# Pythonの基本ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ファイル操作
import os
import glob

# YoutubeAPIの利用
from apiclient.discovery import build
from apiclient.errors import HttpError

# Jupyter上にHTMLを表示する
from IPython.display import HTML

In [3]:
# trainから動画を取得する
file_select = 'okakoro'
all_videos = pd.read_csv('output/train/'+file_select+'/videos.csv', index_col=0)
all_songs = all_videos[all_videos['Label'] == 1]

print("全動画数:", all_songs.shape[0])
all_songs.head(2)

全動画数: 17


,Id,Name,ChannelId,Date,Title,Thumbnail,CategoryId,Duration,DurationOriginal,Description,ViewCount,LikeCount,DislikeCount,CommentCount,Label
11,JqwKeddXPOg,猫又おかゆ,UCvaTdHTWBGv3MKj3KVqJVCw,2021-07-09T12:00:15Z,p.h. / 猫又おかゆ(cover),https://i.ytimg.com/vi/JqwKeddXPOg/hqdefault.jpg,20,154,PT2M34S,＿\n\n\n『p.h.』\n本家様▶https://youtu.be/AKQWRnZ_Fc...,3597013,103314,0,1692,1
12,G49bAinM6Lc,戌神ころね,UChAnqc_AY5_I3Px5dig3X1Q,2020-02-22T12:15:11Z,恋愛裁判 Ver.戌神ころね,https://i.ytimg.com/vi/G49bAinM6Lc/hqdefault.jpg,22,222,PT3M42S,本家様：\nhttps://www.youtube.com/watch?v=TXzfQ0cP...,7745099,175007,0,4141,1


In [4]:
# Youtube APIの準備

# APIキーをファイルから取得
f = open('secret/apikey', 'r')
api_key = f.read()
f.close()

# APIキーを用いてリクエスト用のクラスを作成
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=api_key)

In [5]:
comments = []
video_ids = np.array(all_songs['Id'])

# チェックポイントから再開する手段も用意しておく
ckpt_ids = 0

# 動画のIDから詳細情報を取得する
# 思ったより処理に時間がかかる（コメントの取得が重い?）
for i, video_id in enumerate(video_ids[ckpt_ids:]):

    count = 0
    channel_ids = []
    nextPageToken = 'search_start'
    # 最新の再生数を取得するために改めて取得する
    try:
        video_detail = youtube.videos().list(
            part = 'snippet,statistics,contentDetails', 
            id = video_id, 
        ).execute()
    except HttpError:
        print('データ参照中にエラーが発生しました')
        break
    
    # 全てのコメントを取得する
    while True:
        if nextPageToken == 'search_start':            
            try:
                comment = youtube.commentThreads().list(
                    part = 'id,snippet', 
                    videoId = video_id, 
                    maxResults = 100
                ).execute()
            except HttpError:
                print('データ参照中にエラーが発生しました')
                break
        else:
            try:
                comment = youtube.commentThreads().list(
                    part = 'id,snippet', 
                    videoId = video_id, 
                    maxResults = 100, 
                    pageToken = nextPageToken
                ).execute()
            except HttpError:
                print('データ参照中にエラーが発生しました')
                break

        items = comment['items']
        for item in items:
            count += 1
            # 'authorChannelId'ない場合は除外する
            if 'authorChannelId' in item['snippet']['topLevelComment']['snippet'].keys():
                channel_id = item['snippet']['topLevelComment']['snippet']['authorChannelId']['value']
                channel_ids.append(channel_id)
        
        # 次のページが存在しない場合終了する
        if 'nextPageToken' in comment.keys():
            nextPageToken = comment['nextPageToken']
        else:
            break
    
    video_snippet = video_detail['items'][0]['snippet']
    video_statistics = video_detail['items'][0]['statistics']
    # snippetから取得
    date = video_snippet['publishedAt']
    channel = video_snippet['channelTitle']
    title = video_snippet['title']
    thumbnail = video_snippet['thumbnails']['high']['url']
    # statisticsから取得
    views = 0
    if 'viewCount' in video_statistics.keys():
        views = video_statistics['viewCount']
    
    channel_ids_string = ','.join(channel_ids)
    comments.append([video_id, channel, date, title, thumbnail, views, channel_ids_string])
    
    # 思ったより処理に時間がかかるのでチェックポイントを挟む
    if i+1 % 10 == 0:
        print(i+1, '/', all_songs.shape[0])
        comments_numpy = np.array(comments)
        comments_pandas = pd.DataFrame(data=comments_numpy, 
                                       columns=['Id', 'Channel', 'Date', 'Title', 'Thumbnail', 'ViewCount', 'CommentIds'])
        comments_pandas.to_csv('output/train/'+file_select+'/comments_ckpt_'+str(ckpt_ids)+'_'+str(ckpt_ids+i)+'.csv')
        # 前のチェックポイントを削除

comments_numpy = np.array(comments)
comments_pandas = pd.DataFrame(data=comments_numpy, 
                               columns=['Id', 'Channel', 'Date', 'Title', 'Thumbnail', 'ViewCount', 'CommentIds'])
comments_pandas.to_csv('output/train/'+file_select+'/comments.csv')
print('データ取得が完了しました')

データ取得が完了しました


In [6]:
# 途中で失敗したのでチェックポイントごとに保存したものを結合
# comments_1 = pd.read_csv('output/train/'+file_select+'/comments_ckpt_?_?.csv', index_col=0)
# comments_2 = pd.read_csv('output/train/'+file_select+'/comments_ckpt_?_?.csv', index_col=0)
# comments = pd.concat([comments_1, comments_2])

comments = pd.read_csv('output/train/'+file_select+'/comments.csv', index_col=0)
comments = comments.reset_index()
comments = comments.drop(['index'], axis=1)
comments.to_csv('output/train/'+file_select+'/comments.csv')

print("全動画数:", comments.shape[0])
# 確認
comments.head(2)

全動画数: 17


,Id,Channel,Date,Title,Thumbnail,ViewCount,CommentIds
0,JqwKeddXPOg,Okayu Ch. 猫又おかゆ,2021-07-09T12:00:15Z,p.h. / 猫又おかゆ(cover),https://i.ytimg.com/vi/JqwKeddXPOg/hqdefault.jpg,3597111,"UC1B0CXYaWdjXrPorG3syFlA,UCOwDr6a-HsQxGLIpK1vQ..."
1,G49bAinM6Lc,Korone Ch. 戌神ころね,2020-02-22T12:15:11Z,恋愛裁判 Ver.戌神ころね,https://i.ytimg.com/vi/G49bAinM6Lc/hqdefault.jpg,7745172,"UCesNuEgizYMsZtXVqUH4Btw,UCW_Ck9VtTjZ3b19jSWqW..."
